<a href="https://colab.research.google.com/github/diliprk/MasterThesis-ML4ACADS/blob/master/CycleGAN_Notes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

## CycleGAN Notes:
Initial trials of basic Cycle Consistent Generative Adversarial Networks on a toy dataset show some positive results and it is hoped that with some further tuning we could generate promising results. The toy dataset is generated by a python script using some constant lab test parameters. We can generate a finite number (`n`) of data points for our toy dataset. From the toy dataset we will pick a subset of points to represent the data from the Lab Test Results (`LTR dataset`) and for the simulation dataset `dataset_Sim` we will pick twice the no. of points of the LTR dataset. The Simulation data points have a systematic error where the correction where `c_f = 1`. Our goal in using CycleGANs is to transfer the characteristics of the LTR data distribution to the Simulation points, just as in [this example](https://dmitryulyanov.github.io/feed-forward-neural-doodle/) where CycleGANs were used to transfer the style of Van Gogh paintings to dog photos.

We will evaluate CycleGANs performance in two cases:
* **Case 1**: The LTR dataset is clean without any outliers or errors and shows a clear increasing monotonic trend.
* **Case 2**: The LTR dataset contains one outlier point, where we manipulate the correction factor of this point (`D == 132`) to an erroneous value.

<table>
    <tr>
        <td><b><center>Case 1</center></b><img src='images/CycleGAN_ToyData_Case1.png'></td>
        <td><b><center>Case 2</center></b><img src='images/CycleGAN_ToyData_Case2.png'></td>
    </tr>
</table>

### Expected Output
The CycleGAN is expected to learn the following:<br>
1. The Simulation points have a systematic error (`c_f = 1`), and though it comes from physics based model the simulation points does not capture the real world lab test data (or represent reality).
2. The Lab Test Results (LTR) data is representative of the real world. Though there is physics also in the LTR data (like linear dependencies: `c_f` $\propto$ `dia`) the LTR data also (as in Case 2 above) is non-linear because of uncertainities and outliers in the data.
and finally, the CycleGANs should synthesize new data (i.e transform the Simulation points to LTR style data) such that it captures the non linearity in the LTR data without being influenced (too much ) by outliers and is also physically consistent(`c_f > 1`).

In both cases, we expect the `c_f` value to be > 1 (as it is in the real world) and the CycleGAN should capture the monotonically increasing trend of `D` vs. `c_f` in the LTR dataset in consideration of the neighborhood of points in the Simulation dataset.In _case 2_ , the CycleGAN output should be least impacted by the outlier point in the LTR dataset and the desired output is represented by the **hand drawn**, dotted green line. In other words, the newly CycleGAN synthesised points should have a lower systematic error compared to the Simulation points.
<tr>        
    <td><img src='images/CycleGAN_ToyProblem_ExpectedOutput.png' style="width: 1000px;"/></td>        
</tr>

## Observations:

The training losses for the two generators and discriminators seem to performing satisfactorily.
<br>
<tr>        
    <td><img src='images/CycleGAN_ToyProblem_Loss.png' style="width: 1000px;"/></td>        
</tr>

In **Case 1** (LTR points without any outliers) the style transfer is easy, the CycleGAN generates new points that captures the linear monotonically increasing function (after training for just 2000 epochs with `batch_size` = 4) as the dataset to learn from (LTR dataset) is smooth and simple, but still we get erroneous correction factors at the start.

In **Case 2** where we have one outlier in the LTR dataset, the non-linear trend is captured to an extent (after training for 20,000 epochs with `batch_size` = 2) though it captures the non-linear shape of the LTR dataset distribution, the output is still not as expected and we sometimes get erroneous correction factors (<1).

<table>
    <tr>
        <td><b><center>Case 1</center></b><img src='images/CycleGAN - Output1.png'></td>
        <td><b><center>Case 2</center></b><img src='images/CycleGAN - Output2.png'></td>
    </tr>
</table>

### i.) Increasing size of LTR dataset
Adding more points to the simulation or LTR dataset reduces the error slightly, but is not necessarily helping in reaching the expected style transfer of the non-linear distribution shape.<br>
<tr>        
    <td><img src='images/CycleGAN - Output3.png' style="width: 700px;"/></td>        
</tr>

### ii.) Variable and Inconsistent Results
We also get slightly different results everytime the script is re-run and the network is retrained for the same points, but (it is hoped) this could be fixed by saving the weights and reloading the same training points<br>
<tr>        
    <td><img src='images/CycleGAN - Output4.png' style="width: 700px;"/></td>        
</tr>

### Suspected Parameters:
Hyper parameter tuning for CycleGANs is a complex process compared to hyperparamter tuning of a normal neural network.There are many parameters which impact the output and performance of the CycleGANs on this toy dataset. Some of the recorded observations are:
* No. of points in datasetB (Simulation dataset) - too many points sometimes hinder the learning and style.
* Nr. of training epochs and `batch_size` also impact the training results.

### Further Planned Evaluation with CycleGANS:
* `update_sim_pool` function based on the `update_image_pool` in the [original paper]((https://github.com/junyanz/CycleGAN/blob/master/util/image_pool.lua)) can be tweaked to get better results
* Investigate other Loss functions for the discriminator like `mse` or `binary cross-entropy`or from SR-GAN.
* Investigate `Physical consistenncy loss` from [LLNL Implementation of CycleGAN](https://github.com/rushilanirudh/icf-jag-cycleGAN/blob/master/modelsv2.py).

### Resources:
* https://github.com/junyanz/CycleGAN
* https://machinelearningmastery.com/how-to-develop-cyclegan-models-from-scratch-with-keras/
* [Exploring Generative Physics Models with Scientific Priors
in Inertial Confinement Fusion](https://arxiv.org/pdf/1910.01666.pdf)

### Discussions:
1. How do we improve the CycleGAN output to synthesise points as in the **Expected  Output**?